In [1]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from scoring import eval_model
from transformers import pipeline
import os


/home/user/miniconda3/envs/ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mode="qlora"
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [3]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            quantization_config=bnb_config, 
                                            device_map="auto",
                                            )


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.23it/s]


In [4]:
target_modules=[
"embed_tokens",
"lm_head",
"q_proj",
"k_proj",
"v_proj",
"o_proj",
"gate_proj",
"up_proj",
"down_proj",
]

In [5]:
for name, param in model.named_parameters():
    print(name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.down_proj.weight
model.layers.2.inp

In [6]:
from peft import LoraConfig, get_peft_model


peft_config = LoraConfig(
        task_type="CAUSAL_LM", inference_mode=False, r=32, lora_alpha=32,
        lora_dropout=0.1,
        #target_modules=target_modules,
    )
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 16,777,216 || all params: 6,755,192,832 || trainable%: 0.24836028248556738


In [7]:
for name, param in model.named_parameters():
    print(name)

base_model.model.model.embed_tokens.weight
base_model.model.model.layers.0.self_attn.q_proj.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.k_proj.weight
base_model.model.model.layers.0.self_attn.v_proj.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.o_proj.weight
base_model.model.model.layers.0.mlp.gate_proj.weight
base_model.model.model.layers.0.mlp.up_proj.weight
base_model.model.model.layers.0.mlp.down_proj.weight
base_model.model.model.layers.0.input_layernorm.weight
base_model.model.model.layers.0.post_attention_layernorm.weight
base_model.model.model.layers.1.self_attn.q_proj.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.1.self_attn.q_pr

In [8]:
pipe= pipeline("text-generation", model=model, tokenizer=tokenizer,max_new_tokens=100)
text="Q: あなたはそのたった一人になれますか。なってくれますか。あなたははらの底から真面目ですか｡ A: "
def ask(pipe,text):
    out=pipe(text)
    out=out[0]["generated_text"][len(text):]
    return out

ask(pipe,text)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'Refor

'あなたはそのたった一人になれますか。なってくれますか。あなたははらの底から真面目ですか。\nNote: The above answers are written in a more informal and conversational style, which is common in Japanese. In more formal situations, the answers might be more polite and respectful.\nIn Japanese, the word "あなた" (anata'

In [9]:
import transformers
from datasets import load_dataset,Dataset
from scoring import generate_prompt
import random

context_list=["Q: あなたはそのたった一人になれますか。なってくれますか。あなたははらの底から真面目ですか｡ A: もし私の命が真面目なものなら、私の今いった事も真面目です"]
data_list=[{"text":i} for i in context_list]
dataset = Dataset.from_dict({"text": [item["text"] for item in data_list]})
train_dataset=dataset.map(lambda samples: tokenizer(samples['text']), batched=True)

Map: 100%|██████████| 1/1 [00:00<00:00, 399.27 examples/s]


In [10]:

tokenizer.pad_token = tokenizer.eos_token
train_args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        warmup_steps=0,
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
    )

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=train_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

ans_dict={}
for i in range(10):
    training_result=trainer.train()
    res=ask(pipe,text)
    ans_dict[i]={"out":res,"loss":training_result.training_loss    }
    print(res)


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,3.135200


あなたは、そのたった一人になれますか。なってくれますか。あなたははらの底から真面目ですか。
Note: The above response is a play on words in Japanese, which is a language that uses a complex system of honorifics and polite language to show respect to the person being spoken to. In this case, the speaker is using a phrase that can


Step,Training Loss
1,2.842000


はらの底から真面目ですか。
This is a humorous example of a question that is nonsensical because the word "はら" (har) is not a word in Japanese. The phrase "はらの底から真面目ですか" (haru no soko kara mae naki desu ka) is a play on words, combining the word "はら" (haru) which


...

Step,Training Loss
1,0.898000


もし私の命が真面目なものなら、私の今日の食事も真面目ですか。私の今日の仕事も真面目ですか。私の今日の私生活も真面目ですか。私の今日の命も真面目ですか。私の今日の今日も真


Step,Training Loss
1,0.697300


/home/user/miniconda3/envs/ft/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


もし私の命が真面目なものなら、私の今いった事も真面目ですから｡ Q: あなたは今日の私たちの真面目さを知っていますか｡ A: 私たちの真面目さは今日の私たちの真面目さですか��


In [11]:
import pandas as pd
pd.DataFrame.from_dict(ans_dict).T

,out,loss
0,あなたは、そのたった一人になれますか。なってくれますか。あなたははらの底から真面目ですか。\...,3.135242
1,はらの底から真面目ですか。\nThis is a humorous example of a...,2.842
2,ありがとうございます。私は、あなたのお願いを聞いたことがあります。私は、あなたのお願いを受け...,2.52715
3,あれは、人生の苦悩を救いますか。Q: あなたは、人生の苦悩を救いますか。A: あれは、人生の...,2.20751
4,真面目ですか。あなたははらの底から真面目ですか。Q: あなたははらの底から真面目ですか。あな...,1.882246
5,それは私の弱点です。私は真面目すぎて、誰にでも寛大です。」\nThis is a humor...,1.608033
6,真面目ですか。あなたははらの底から真面面ですか。 Q: あなたは真面目ですか。あなたははらの...,1.36228
7,もし私の真面目さが私の為にあるなら、今度私の命を壊すことができるでしょうか。私の真面目さは私...,1.132123
8,もし私の命が真面目なものなら、私の今日の食事も真面目ですか。私の今日の仕事も真面目ですか。私...,0.898009
9,もし私の命が真面目なものなら、私の今いった事も真面目ですから｡ Q: あなたは今日の私たちの...,0.697253
